In [1]:
import numpy as np
import pandas as pd
import re
# import os
# import codecs

In [2]:
target_url = ("/Users/devkrroy/Dev/Projects/snow/data/export3.xlsx")

In [3]:
df = pd.read_excel(target_url)

In [4]:
print(df.columns.values)

['Number' 'Assignment_group' 'Priority' 'State' 'Category' 'Subcategory'
 'Severity' 'Closure_CI' 'Location' 'Close_code' 'Configuration_item'
 'Child_Incidents' 'Contact_type' 'Duration' 'Impact' 'Short_description'
 'Created' 'dates' 'date' 'day' 'month' 'dom' 'week' 'week2' 'cluster'
 'cluster1' 'cluster_name1' 'cluster_name']


In [5]:
features = ['Number', 'Assignment_group', 'Priority', 'State', 'Category',
       'Subcategory', 'Severity', 'Closure_CI', 'Location', 'Close_code',
       'Configuration_item', 'Child_Incidents', 'Contact_type',
       'Duration', 'Impact', 'Short_description', 'Created']
df = df[features]

In [6]:
print("Shape of input file", df.shape)
# df = df.head(2000)

Shape of input file (95373, 17)


In [7]:
#Filter out some categories with instances < 50 for now , remove this filter in future
# df = df.groupby("Category").filter(lambda x: len(x) > 50)
# df = df.reset_index(drop =True)
# df.Category.value_counts()

In [8]:
# nltk text preprocessing
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import contractions

from sklearn import feature_extraction
# from nltk.tag import pos_tag

stop = stopwords.words('english')
lemmer=WordNetLemmatizer()

#extend stop words
new_stopwords = ['chw','edu','phx','vapp']
stop.extend(new_stopwords)

# to be tried - apply text collocations

def pre_process(text):
        
    # try to capture the IPs    
    ip_pattern = re.compile(r'([A-Za-z0-9]{2,}[-.]{1,1}){2,6}([A-Za-z0-9]*)')   
    ip = text.apply(lambda x: ' '.join(word for word in re.split('[\': (),]', x) if ip_pattern.match(word)))
    
    text = text.apply(lambda x: ' '.join(word.lower() for word in x.split()))  # lower case
#   text = text.apply(lambda x: ' '.join(contractions.fix(word) for word in x.split()))  # replace contractions

    text = text.str.replace('[^A-Za-z ]', ' ')  # remove non words
    
    text = text.apply(lambda x: ' '.join([word for word in x.split() if word not in set(stop)]))   #stop
    text = text.apply(lambda x: ' '.join([lemmer.lemmatize(word, pos='v') for word in x.split()]))       #lemma
    text = text.apply(lambda x: ' '.join([word for word in x.split() if len(word) > 1 ]))  #length >1

    return (text, ip)
    
normalized = pre_process(df['Short_description'])

In [9]:
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize

tokenized_word=word_tokenize(normalized[0].to_string())

fdist = FreqDist(tokenized_word)
print(fdist)

<FreqDist with 119470 samples and 736994 outcomes>


In [10]:
fdist.most_common(50)

[('...', 43458),
 ('issue', 18550),
 ('unable', 16168),
 ('network', 13663),
 ('login', 12342),
 ('id', 11318),
 ('account', 10722),
 ('user', 10677),
 ('utilization', 9289),
 ('lock', 6667),
 ('set', 6318),
 ('desktop', 6288),
 ('threshold', 6064),
 ('nmscust', 5907),
 ('password', 5889),
 ('cerner', 5699),
 ('need', 5684),
 ('printer', 5671),
 ('mdf', 5595),
 ('reset', 5240),
 ('monitor', 4601),
 ('phone', 4516),
 ('device', 4365),
 ('cpu', 4228),
 ('able', 4222),
 ('ms', 4202),
 ('main', 4135),
 ('keyboard', 4045),
 ('pc', 4031),
 ('dev', 3647),
 ('application', 3633),
 ('ap', 3630),
 ('username', 3185),
 ('computer', 3170),
 ('helpdesk', 3005),
 ('zother', 2973),
 ('access', 2659),
 ('file', 2642),
 ('error', 2631),
 ('rt', 2577),
 ('mouse', 2505),
 ('work', 2485),
 ('wc', 2401),
 ('up', 2392),
 ('smart', 2364),
 ('rid', 2353),
 ('duo', 2258),
 ('kyocera', 2244),
 ('job', 2196),
 ('xl', 2182)]

In [11]:
df['Terms'] = normalized[0]
df['IPs'] = normalized[1]

In [12]:
df['IPs'][:10]

0    phx-vapp-689.chw.edu
1                        
2            PHX-VAPP-408
3                        
4                        
5                        
6                        
7           10.203.212.24
8    phx-vapp-842.chw.edu
9                        
Name: IPs, dtype: object

In [13]:
# import re

# print(re.split('[:\' ]', '\'phx-vapp-689.chw.edu\': Utilization is \'85.2\'. Threshold is set to \'Disk/File System/{C}/percent full >= 85\''))

# ci_pattern = re.compile(r'([A-Za-z0-9]{2,}[-.]{1,1}){2,6}([A-Za-z0-9]*)')

# print(ci_pattern.match('phx-vapp-238.chw.edu'))
# print(ci_pattern.match('CPU Utilization is '))
# print(ci_pattern.match('85.2'))
# print(ci_pattern.match('10.250.243.2'))
# print(ci_pattern.match('192.168.2.85)'))
# test = pd.DataFrame({'strings': ['...', 'a(b$c']})
# test.strings.str.replace('[^a-zA-Z ]', ' ')

In [14]:
# define a tokenizer which returns the set of tokens in the text that it is passed

def tokenize_only(text):
    # tokenize by  word 
    tokens = [word.lower() for word in nltk.word_tokenize(text) ]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer( max_features=10000, max_df=0.8, min_df=0.0005,                                  
                                  use_idf=True, tokenizer=word_tokenize, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(df['Terms'].values.astype('U'))

print(tfidf_matrix.shape)

CPU times: user 13.3 s, sys: 89.6 ms, total: 13.4 s
Wall time: 13.4 s
(95373, 3737)


In [27]:
terms = tfidf_vectorizer.get_feature_names()
print(terms[100:200])

['activate issue', 'activate issue resolve', 'activation', 'activation code', 'activation link', 'activation link user', 'active', 'active job', 'active job complete', 'ad', 'adapter', 'add', 'add printer', 'additional', 'additional info', 'additional info time', 'address', 'ade', 'adg', 'adg pck', 'adg pck bla', 'adg rid', 'adg rid bla', 'adg wealthpoint', 'adg wealthpoint ontime', 'adh', 'admin', 'admin access', 'admin mdf', 'admin mdf main', 'admin right', 'admin right need', 'administration', 'administration run', 'admit', 'admit loc', 'adobe', 'adt', 'adt allergy', 'adt issue', 'adt nitiate', 'adt nitiate identity', 'adt order', 'adt source', 'adt source ini', 'adt st', 'agch', 'agency', 'agency tier', 'agency tier mft', 'agent', 'agent run', 'agg', 'agg mdf', 'agg mdf dt', 'agg mdf main', 'agg mdf sjpvh', 'alarm', 'alarm js', 'alarm js nberrorcond', 'alarm nberrorcond', 'alarm qs', 'alarm qs nberrorcond', 'alert', 'alert dhi', 'alert dhi ssrm', 'alert high', 'alive', 'alive error

In [28]:
# Mini Batch K means
from sklearn.cluster import MiniBatchKMeans
num_clusters = 30
km = MiniBatchKMeans(n_clusters = num_clusters, init = 'k-means++', n_init = 3, init_size = 300, batch_size = 100, random_state = 1)
%time km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

CPU times: user 1.06 s, sys: 3.93 ms, total: 1.07 s
Wall time: 1.07 s


In [29]:
# Save cluster in pickle file for future use
# import pickle

# pickle.dump(km, open('pkl/km_1.pkl', 'wb'))

In [30]:
# Create mapping of ngram features into clusters, 
# save into terms map object with key as cluster name (first feature in each cluster)
# terms = top 20 ngram features

order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names()
terms_map = pd.DataFrame(index=np.arange(0, num_clusters),columns=['terms','key'])    
for i in range(num_clusters):
    print("Cluster %d:" % i, end='')
    #terms_map['key'] = i
    list = [] 
    for ind in order_centroids[i, :20]:
        print(' %s' % terms[ind], end=',')
        list.append(terms[ind])
    for ind2 in order_centroids[i, :1]:
        terms_map.key.loc[i] = terms[ind2]

    str = ', '.join(list) 
    terms_map.terms.loc[i] = str
    print()
print(terms_map.shape)

Cluster 0: lock user, account lock user, lock user id, account, lock, account lock, user, user id, network account, network account lock, network, id, unlock account, unlock, account issue, unlock account issue, issue, issue resolve, resolve, account issue resolve,
Cluster 1: threshold set disk, disk file, set disk, set disk file, disk file system, percent full, file system, disk, percent, full, system, file system percent, system percent, system percent full, file, utilization threshold set, threshold set, utilization threshold, utilization, set,
Cluster 2: login, unable login, unable, login username, unable login username, username, network, network unable, network unable login, account, account unable, network account unable, account unable login, do, encryption unable login, encryption, encryption unable, password, network account, unable login uid,
Cluster 3: utilization, threshold set utilization, set utilization, cpu utilization threshold, cpu utilization, cpu, utilization thres